Mount google drive 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
ls "/content/drive/MyDrive/SDE Project/Datasets/smart-meters-in-london/daily_dataset.csv"

'/content/drive/MyDrive/SDE Project/Datasets/smart-meters-in-london/daily_dataset.csv'


In [8]:
!pip install great_expectations

     |████████████████████████████████| 5.0 MB 4.3 MB/s 
     |████████████████████████████████| 793 kB 49.3 MB/s 
     |████████████████████████████████| 3.6 MB 46.7 MB/s 
     |████████████████████████████████| 503 kB 56.2 MB/s 
     |████████████████████████████████| 109 kB 55.1 MB/s 
     |████████████████████████████████| 67 kB 5.7 MB/s 
     |████████████████████████████████| 9.9 MB 44.5 MB/s 
     |████████████████████████████████| 138 kB 60.9 MB/s 
     |████████████████████████████████| 381 kB 67.0 MB/s 
     |████████████████████████████████| 428 kB 55.4 MB/s 
     |████████████████████████████████| 127 kB 60.5 MB/s 
     |████████████████████████████████| 546 kB 56.7 MB/s 
  Attempting uninstall: tornado
    Found existing installation: tornado 5.1.1
    Uninstalling tornado-5.1.1:
      Successfully uninstalled tornado-5.1.1
  Attempting uninstall: prompt-toolkit
    Found existing installation: prompt-toolkit 1.0.18
    Uninstalling prompt-toolkit-1.0.18:
      Successfull

In [9]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# install spark (change the version number if needed)
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz

# unzip the spark file to the current folder
!tar xf spark-3.0.0-bin-hadoop3.2.tgz

# set your spark folder to your system path environment. 
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"


# install findspark using pip
!pip install -q findspark

In [10]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 34 kB/s 
     |████████████████████████████████| 198 kB 68.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=fd08fea7e3d4bc1455ddab5d2f2dbded53024f91562d0909829005fa063c8e90
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [11]:
import findspark
findspark.init()

In [12]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [13]:
df = spark.read.format("csv").option("header","true").load("/content/drive/MyDrive/SDE Project/Datasets/smart-meters-in-london/daily_dataset.csv")

In [14]:
df.show()

+---------+----------+-------------------+-------------------+------------------+------------+-------------------+------------------+-------------------+
|    LCLid|       day|      energy_median|        energy_mean|        energy_max|energy_count|         energy_std|        energy_sum|         energy_min|
+---------+----------+-------------------+-------------------+------------------+------------+-------------------+------------------+-------------------+
|MAC000131|2011-12-15|              0.485|0.43204545454545457|             0.868|          22|0.23914579678767536|             9.505|0.07200000000000001|
|MAC000131|2011-12-16|             0.1415|0.29616666875000003|         1.1160001|          48| 0.2814713178628203|14.216000100000002|              0.031|
|MAC000131|2011-12-17|             0.1015|          0.1898125|             0.685|          48| 0.1884046862418033|             9.111|              0.064|
|MAC000131|2011-12-18|              0.114| 0.2189791666666666|0.675999999999

In [15]:
# creating GE wrapper around spark dataframe
from great_expectations.dataset.sparkdf_dataset import SparkDFDataset
gdf = SparkDFDataset(df) 

# gdf holds referance to df as gdf.spark_df
# we can access the df as gdf.spark_df

gdf.spark_df.show(10)

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


+---------+----------+-------------------+-------------------+------------------+------------+-------------------+------------------+-------------------+
|    LCLid|       day|      energy_median|        energy_mean|        energy_max|energy_count|         energy_std|        energy_sum|         energy_min|
+---------+----------+-------------------+-------------------+------------------+------------+-------------------+------------------+-------------------+
|MAC000131|2011-12-15|              0.485|0.43204545454545457|             0.868|          22|0.23914579678767536|             9.505|0.07200000000000001|
|MAC000131|2011-12-16|             0.1415|0.29616666875000003|         1.1160001|          48| 0.2814713178628203|14.216000100000002|              0.031|
|MAC000131|2011-12-17|             0.1015|          0.1898125|             0.685|          48| 0.1884046862418033|             9.111|              0.064|
|MAC000131|2011-12-18|              0.114| 0.2189791666666666|0.675999999999

In [16]:
from great_expectations.profile.basic_dataset_profiler import BasicDatasetProfiler

expectation_suite_based_on_profiling, validation_result_based_on_profiling = gdf.profile(BasicDatasetProfiler)

In [17]:
# visualize the profiling result and expectation came from profiling

# import the renderer who will basically create the document content
from great_expectations.render.renderer import ProfilingResultsPageRenderer, ExpectationSuitePageRenderer
# import the view template who will basically convert the document content to HTML
from great_expectations.render.view import DefaultJinjaPageView

profiling_result_document_content = ProfilingResultsPageRenderer().render(validation_result_based_on_profiling)
# it is of type great_expectations.render.types.RenderedDocumentContent
expectation_based_on_profiling_document_content = ExpectationSuitePageRenderer().render(expectation_suite_based_on_profiling)
# it is also of type great_expectations.render.types.RenderedDocumentContent

In [18]:
# we will generate the HTML
profiling_result_HTML = DefaultJinjaPageView().render(profiling_result_document_content) # type string or str
expectation_based_on_profiling_HTML = DefaultJinjaPageView().render(expectation_based_on_profiling_document_content)

/usr/local/lib/python3.7/dist-packages/jinja2/environment.py:1088: DeprecationWarning: 'soft_unicode' has been renamed to 'soft_str'. The old name will be removed in MarkupSafe 2.1.
  return concat(self.root_render_func(self.new_context(vars)))


In [19]:
file = open("/tmp/sample.html","w")
file.write(profiling_result_HTML)
file.close()

In [20]:
import IPython
IPython.display.HTML(filename='/tmp/sample.html')

,
Number of variables,9
Number of observations expect_table_row_count_to_be_between,3510433
Missing cells,0.04%
,
int,0
float,0
string,9
datetime,0
bool,0
unknown,0
